In [1]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
import sys, glob, math, os
import cv2
import numpy as np
import re

# image sizes
img_rows, img_cols = 224, 224

Using TensorFlow backend.


In [2]:
def vgg19_lower_model():
    # initial model, load vgg19
    vgg19 = VGG19(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
    vgg19.summary()

    # new model that takes first 10 layers of VGG19
    model = Sequential()
    model.add(vgg19.get_layer('input_1'))
    model.add(vgg19.get_layer('block1_conv1'))
    model.add(vgg19.get_layer('block1_conv2'))
    model.add(vgg19.get_layer('block1_pool'))
    model.add(vgg19.get_layer('block2_conv1'))
    model.add(vgg19.get_layer('block2_conv2'))
    model.add(vgg19.get_layer('block2_pool'))
    model.add(vgg19.get_layer('block3_conv1'))
    model.add(vgg19.get_layer('block3_conv2'))
    model.add(vgg19.get_layer('block3_conv3'))
    model.add(vgg19.get_layer('block3_conv4'))
    model.add(vgg19.get_layer('block3_pool'))
    model.add(vgg19.get_layer('block4_conv1'))
    model.add(vgg19.get_layer('block4_conv2'))
    model.add(vgg19.get_layer('block4_conv3'))
    model.add(vgg19.get_layer('block4_conv4'))
    model.add(vgg19.get_layer('block4_pool'))
#    model.add(vgg19.get_layer('block5_conv1'))
#    model.add(vgg19.get_layer('block5_conv2'))
#    model.add(vgg19.get_layer('block5_conv3'))
#    model.add(vgg19.get_layer('block5_conv4'))
#    model.add(vgg19.get_layer('block5_pool'))
#    model.add(Flatten())
    model.summary()
 
    return model
#END vgg19_lower_model()

In [3]:
def load_imgs(dir,normalize=0):
    # load image data from dir

    data = []
    fname = []
    path  = dir + '/scene*.JPG'
    files = sorted(glob.glob(path))

    for fl in files:
        flbase = os.path.basename(fl)
        print('reading %s'%format(fl))

        img = cv2.imread(fl)
        img = cv2.resize(img, (img_rows,img_cols))
        img = np.array(img, dtype=np.float32)
        if normalize == 1:
            img -= np.mean(img)
            img /= np.std(img)

        data.append(img)
        fname.append(fl)

    data = np.array(data,dtype=np.float32)

    return data, fname
# end of load_imgs

In [4]:
def get_id_from_fname(str):
    
    m = re.match(r"([a-z/]+)([0-9]+)([a-z]+).[a-zA-Z]",str)

    if m:
    #    print("found!")
        id = int(m.group(2))
    #    print(id)
    else:
    #    print("not found!")
        id = -1
    
    return id
#end of get_id_from_frame

In [5]:
def save_array_csv(fname,val):

    f = open(fname, 'w')

    for i in range(0,val.shape[0]):
        for j in range(0,val.shape[1]):
            for k in range(0,val.shape[2]):
                f.write("%d, %d, %d, "%(i,j,k)) 
                print("%d, %d, %d, "%(i,j,k)) 
                for l in range(0,val.shape[3]):
                    f.write("%f, "%(val[i][j][k][l]))
                f.write("\n") 

    f.close()
#end of save_array_csv

In [6]:
if __name__=='__main__':

    data,fname = load_imgs('scenes',1) 
    print(data.shape)

reading scenes/scene007a.JPG
reading scenes/scene007b.JPG
reading scenes/scene007c.JPG
reading scenes/scene007d.JPG
reading scenes/scene007e.JPG
reading scenes/scene007f.JPG
reading scenes/scene007g.JPG
reading scenes/scene008a.JPG
reading scenes/scene008b.JPG
reading scenes/scene009a.JPG
reading scenes/scene009b.JPG
reading scenes/scene009c.JPG
reading scenes/scene009d.JPG
reading scenes/scene009e.JPG
reading scenes/scene009f.JPG
reading scenes/scene010a.JPG
reading scenes/scene010b.JPG
reading scenes/scene010c.JPG
reading scenes/scene010d.JPG
reading scenes/scene011a.JPG
reading scenes/scene011b.JPG
reading scenes/scene011c.JPG
reading scenes/scene012a.JPG
reading scenes/scene012b.JPG
reading scenes/scene012c.JPG
reading scenes/scene012d.JPG
reading scenes/scene013a.JPG
reading scenes/scene013b.JPG
reading scenes/scene013c.JPG
reading scenes/scene013d.JPG
reading scenes/scene014a.JPG
reading scenes/scene014b.JPG
reading scenes/scene014c.JPG
reading scenes/scene014d.JPG
reading scenes

In [7]:
    model = vgg19_lower_model()
    model.summary()
    out = model.predict(data, batch_size=32, verbose=1)

    print(out.shape)

ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096]
	 [[Node: fc1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4048235, _device="/job:localhost/replica:0/task:0/gpu:0"](fc1/random_uniform/shape)]]

Caused by op 'fc1/random_uniform/RandomUniform', defined at:
  File "/home/nakazawa/anaconda3/envs/py3-tf2-keras2/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/nakazawa/anaconda3/envs/py3-tf2-keras2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nakazawa/anaconda3/envs/py3-tf2-keras2/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/nakazawa/anaconda3/envs/py3-tf2-keras2/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/nakazawa/anaconda3/envs/py3-tf2-keras2/lib/python3.5/site-packages/ipykernel

In [8]:
#    save_array_csv("out.csv", out)
#    np.save("out.npy",out)

In [9]:
from scipy.cluster.vq import vq, kmeans, whiten
from sklearn.preprocessing import StandardScaler

data = np.reshape(out,(out.shape[0],out.shape[1]*out.shape[2],out.shape[3]))
print(data.shape)

(184, 196, 512)


In [10]:
# data standlization
data_for_cluster = np.reshape(data,(data.shape[0]*data.shape[1],data.shape[2]))

# standlization step
me = np.mean(data_for_cluster,0)
stv = np.std(data_for_cluster - me,0)
#stv2 = np.std(data_for_cluster,0)
#print(stv)
#print(stv2)
data_standlized = (data_for_cluster - me) / stv
data_whitened = data_standlized

In [11]:
# whiteneing step
#data_whitened = whiten(data_for_cluster)
#print(data_whitened)
#print(np.std(data_whitened,0))
centers, distortion = kmeans(data_whitened,200)
print(centers)
print(distortion)
print(data_whitened.shape)

[[ 0.44781414  0.06399728 -0.16210803 ..., -0.19624718  0.29379928
   0.35572395]
 [-0.26115152  0.2545622   0.0195284  ..., -0.2536979   0.93594474
  -0.24854407]
 [ 0.08019625 -0.26907921 -0.21473019 ..., -0.27616391  0.22975214
   0.65544939]
 ..., 
 [-0.26356417 -0.06681782 -0.23421821 ...,  0.11267113  0.03093067
   0.06003801]
 [-0.26848403 -0.12055506 -0.0354185  ...,  0.0983579  -0.13180141
  -0.20948291]
 [-0.26217619 -0.07172839 -0.19674128 ..., -0.38899919 -0.33530885
  -0.3005985 ]]
18.1327
(36064, 512)


In [12]:
#data = np.random.rand(100,5)
#standardized_data = StandardScaler().fit_transform(data)
#print(np.mean(standardized_data,0))

In [13]:
#np.save("data_whitened.npy",data_whitened)
np.save("data",data)
np.save("centers.npy",centers)
np.save("mean.npy",me)
np.save("stv.npy",stv)